In [ ]:
!pip install openai python-dotenv pypdf gradio

In [2]:
import os
import uuid
import pandas as pd
import gradio as gr
from dotenv import load_dotenv
from supabase import create_client, Client
from openai import OpenAI

In [3]:
# Load environment variables
load_dotenv(override=True)

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Validate keys
if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("Supabase URL or Key not found in environment variables")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")

# Initialize clients
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
openaiapi = os.getenv("OPENAI_API_KEY")
groqapi = os.getenv("groq_api_key")


if openaiapi:
    print(f"Openai key is found and starts with: {openaiapi[:8]}")
else:
    print("Open ai api not found")

if groqapi:
    print(f"groqapi key is found and starts with: {groqapi[:8]}")
else:
    print("groqapi ai api not found")

Openai key is found and starts with: sk-proj-
groqapi key is found and starts with: gsk_Vopn


In [5]:
# System prompt for property recommendations
system_prompt_property = """
You are a real estate advisor AI. 
You are given a dataset of properties including location, price, bedrooms, bathrooms, parking spaces, toilets, title, and description. 
When a user provides their preferences (budget, location, property type, number of bedrooms), you must return the top 3 properties that best match their needs. 
Provide a short reasoning for each recommendation. 
Always consider price affordability and feature match.
"""

In [6]:
# Load property dataset
sheet_url = "https://docs.google.com/spreadsheets/d/15h3j-Q-Xepsok2ru5Au_havdWMdJ983qBGBXzceCuig/export?format=csv&gid=474984245"
df_cleaned = pd.read_csv(sheet_url)


In [7]:
df_cleaned

,Location,Currency,Price,Contact,Bedrooms,Realtor,Bathrooms,Title,Description,Parking Spaces,Toilets,Score
0,"Urban Prime Three Phase 2 Estate, Ogombo, Ajah...",₦,1.500000e+08,8067495852,5,Lake Pad Ventures,6,Luxury 5 Bedrooms Semi-detached Duplex,Hurry now!\nfor sale\nluxury 5 bedroom on two ...,5,7,22.0
1,"Ikoyi, Lagos",₦,4.290552e+09,7065266133,6,Bera Deals,6,Luxury 6 Bedrooms Penthouse Condo,Direct brief from the owner\n- the detailed vi...,0,7,21.5
2,"Pinnock Beach Estate, Osapa, Lekki, Lagos",₦,1.400000e+09,7065266133,6,Bera Deals,4,Newly Built Massive House / Mansion,Direct brief / direct instruction from the own...,0,7,19.5
3,"Oseni Mayegun Street, Ologolo, Lekki, Lagos",₦,2.000000e+08,8079792003,4,Paul Osaji And Co,4,4 Bedroom Detached House With 2 Mini Flats Bq,For sale in ologolo\n4 bedroom fully detached ...,0,4,14.0
4,"Ikate Lekki, Ikate, Lekki, Lagos",₦,2.100000e+08,8084677480,5,Jasmine Real Properties And Estate Services,5,Super Luxury 5bed Semi Detach Duplex With A Bq,5 bed ikate n215m\nluxury 5 bedroom semidetach...,0,6,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...
50242,"Tulip Heaven, Chevron, Lekki Phase 1, Lekki, L...",₦,2.600000e+08,8033012341,5,First Close,6,Newly Built 5 Bedrooms Fully Detached House,"Holiday price now n260,000,000.00 from initial...",0,7,19.5
50243,"Aguda, Surulere, Lagos",₦,1.500000e+08,8054563213,4,Harris Consult Realty,4,Renovated Block Of 16 Flats,Direct brief\nfor sale\n16 flat comprised of 9...,0,5,14.5
50244,"Katampe Extension, Katampe, Abuja",₦,2.500000e+09,8155321626,9,Akerejolabisoye Global Ltd,10,Newly Built Luxury 9 Bedroom Mansion With 2 Be...,Newly built luxury 9 bedroom mansion with 2 be...,10,10,38.0
50245,"Banana Island, Ikoyi, Lagos",₦,6.129360e+09,9016641836,5,Godmaan Global Realestate,8,"Convenient, Contemporary Living With Treetop V...","Located in a sought-after neighborhood, this p...",8,8,26.0


In [8]:
# Logging functions
def log_interaction(session_id, user_message, bot_reply):
    try:
        supabase.table("interactions").insert({
            "session_id": session_id,
            "user_message": user_message,
            "bot_reply": bot_reply
        }).execute()
    except Exception as e:
        print(f"[Log Interaction Error]: {e}")

def log_filters(session_id, budget, location, property_type, bedrooms):
    try:
        supabase.table("filters").insert({
            "session_id": session_id,
            "budget": budget,
            "location": location,
            "property_type": property_type,
            "bedrooms": bedrooms
        }).execute()
    except Exception as e:
        print(f"[Log Filters Error]: {e}")

def log_recommendations(session_id, df_top, limit=3):
    try:
        recs = df_top.sort_values(by="Score", ascending=False).head(limit)
        payload = []
        for _, r in recs.iterrows():
            payload.append({
                "session_id": session_id,
                "property_title": str(r.get("Title", "")),
                "price": float(r.get("Price", 0) or 0),
                "location": str(r.get("Location", "")),
                "score": float(r.get("Score", 0) or 0),
            })
        if payload:
            supabase.table("recommendations").insert(payload).execute()
    except Exception as e:
        print(f"[Log Recommendations Error]: {e}")

# Core functions
def get_recommendation_input(budget=None, location=None, property_type=None, bedrooms=None):
    """Filter dataset and prepare prompt for LLM"""
    df_filtered = df_cleaned.copy()

    if budget is not None:
        df_filtered = df_filtered[df_filtered['Price'] <= budget]
    if location:
        df_filtered = df_filtered[df_filtered['Location'].str.contains(location, case=False, na=False)]
    if property_type:
        df_filtered = df_filtered[df_filtered['Title'].str.contains(property_type, case=False, na=False)]
    if bedrooms is not None:
        df_filtered = df_filtered[df_filtered['Bedrooms'] == bedrooms]

    df_top = df_filtered.sort_values(by='Score', ascending=False).head(5)

    prompt = f"Recommend top properties based on these preferences:\nBudget: {budget}\n"
    prompt += f"Location: {location}\nProperty Type: {property_type}\nBedrooms: {bedrooms}\n\n"
    prompt += "Available properties:\n"
    for idx, row in df_top.iterrows():
        prompt += f"- {row['Title']} | Price: ₦{row['Price']:,.0f} | Location: {row['Location']} | Bedrooms: {row['Bedrooms']} | Bathrooms: {row['Bathrooms']} | Parking: {row['Parking Spaces']} | Toilets: {row['Toilets']} | Score: {row['Score']}\n"

    return prompt, df_top

def chat_property(user_message, history):
    """Send conversation to OpenAI API with chat history"""
    messages = [{"role": "system", "content": system_prompt_property}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})
    messages.append({"role": "user", "content": user_message})

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"[Chat API Error]: {e}")
        return "Sorry, I couldn't process your request at the moment."

def recommend_with_filters(budget, location, property_type, bedrooms, history=None, session_id=None):
    """Handles filtering, chat, and logging"""
    if history is None:
        history = []

    if session_id is None:
        session_id = str(uuid.uuid4())

    user_prompt, df_top = get_recommendation_input(budget, location, property_type, bedrooms)
    reply = chat_property(user_prompt, history)
    history.append((user_prompt, reply))

    # Log to Supabase
    log_filters(session_id, budget, location, property_type, bedrooms)
    log_recommendations(session_id, df_top)
    log_interaction(session_id, user_prompt, reply)

    return history, history, session_id

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🏡 Property Finder Chatbot")

    with gr.Row():
        budget = gr.Number(label="Budget (₦)", value=200000000)
        location = gr.Textbox(label="Location", placeholder="e.g., Lagos")
        property_type = gr.Textbox(label="Property Type", placeholder="e.g., Apartment")
        bedrooms = gr.Number(label="Bedrooms", value=3)

    chatbot = gr.Chatbot()
    state = gr.State([])
    session_state = gr.State(None)

    send_button = gr.Button("Find Properties")

    send_button.click(
        recommend_with_filters,
        inputs=[budget, location, property_type, bedrooms, state, session_state],
        outputs=[chatbot, state, session_state]
    )

demo.launch()

C:\Users\Admin\AppData\Local\Temp\ipykernel_12120\1033509750.py:112: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
